In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
jr = pd.read_csv('data/gmonpro_20221207_090716_Sim1_44011.csv', header=None)

In [4]:
jr.head(3)

,0
0,PLMN;SYSTEM;XCI;xNBID;LOCAL_CID;LAC/TAC;PCI/PS...
1,44011;4;71726856;280183;8;211;251;1575;1800 B3...
2,44011;4;71726856;280183;8;211;251;1575;1800 B3...


In [5]:
jr = jr[0].str.split(';', expand=True)
jr.columns = jr.iloc[0]
jr.drop(0, inplace=True)
jr.reset_index(drop=True, inplace=True)
jr.rename(columns={'PCI/PSC/BSIC':'PCI', 'RSRP/RSCP':'RSRP', 'RSRQ/ECIO':'RSRQ', 'SNR':'SINR', 'LAT':'LAT_org', 'LON':'LON_org', 'TIME':'TIME_org'}, inplace=True)
jr.insert(0, 'LON', jr['LON_org'])
jr.insert(0, 'LAT', jr['LAT_org'])
jr.insert(0, 'TIME', jr['TIME_org'])
jr.drop(['LON_org', 'LAT_org', 'TIME_org'], axis=1, inplace=True)

In [6]:
jr.head(3)

,TIME,LAT,LON,PLMN,SYSTEM,XCI,xNBID,LOCAL_CID,LAC/TAC,PCI,ARFCN,BAND,RSSI,RSRP,RSRQ,SINR,CQI,TA,DISTANCE,DELTA_AZI,SPEED,GPS_ACCURACY,UL,DL,BANDWIDTH,BANDWIDTHS,CA,NR_STATE,NARFCN,NR_BAND,NR_PCI,NR_SS_RSRP,NR_SS_RSRQ,NR_SS_SINR,NR_CSI_RSRP,NR_CSI_RSRQ,NR_CSI_SINR,CLF_LABEL,CLF_LOC,CLF_DESC,DATE,ROAMING
0,09:07:16,35.698181,139.773952,44011,4,71726856,280183,8,211,251,1575,1800 B3,-65,-92,-6,1.0,,,,,0,25,22,120,5000,5,1,none,,,,,,,,,,--,--,Cell not found in the clf database!,2022/12/07,HOME
1,09:07:17,35.698181,139.773952,44011,4,71726856,280183,8,211,251,1575,1800 B3,-65,-92,-6,1.0,,56,,,0,25,11,9,5000,5,1,none,,,,,,,,,,--,--,Cell not found in the clf database!,2022/12/07,HOME
2,09:07:18,35.698181,139.773952,44011,4,71726856,280183,8,211,251,1575,1800 B3,-65,-92,-6,1.0,,56,,,0,25,69,27,5000,5,1,none,,,,,,,,,,--,--,Cell not found in the clf database!,2022/12/07,HOME


In [7]:
jr[['xNBID', 'PCI']].value_counts()

xNBID   PCI
279397  33     110
280189  135     66
279397  158     23
280183  251     22
280070  135     19
279617  75      18
279592  260     17
279398  74      17
280209  317     17
279041  95      16
279592  241     10
280183  15      10
280189  322     10
279398  7        7
279394  202      7
279633  190      5
        313      4
279770  110      3
279394  199      3
278920  365      2
279617  332      2
279041  123      2
dtype: int64

In [8]:
jr.columns

Index(['TIME', 'LAT', 'LON', 'PLMN', 'SYSTEM', 'XCI', 'xNBID', 'LOCAL_CID',
       'LAC/TAC', 'PCI', 'ARFCN', 'BAND', 'RSSI', 'RSRP', 'RSRQ', 'SINR',
       'CQI', 'TA', 'DISTANCE', 'DELTA_AZI', 'SPEED', 'GPS_ACCURACY', 'UL',
       'DL', 'BANDWIDTH', 'BANDWIDTHS', 'CA', 'NR_STATE', 'NARFCN', 'NR_BAND',
       'NR_PCI', 'NR_SS_RSRP', 'NR_SS_RSRQ', 'NR_SS_SINR', 'NR_CSI_RSRP',
       'NR_CSI_RSRQ', 'NR_CSI_SINR', 'CLF_LABEL', 'CLF_LOC', 'CLF_DESC',
       'DATE', 'ROAMING'],
      dtype='object', name=0)

In [9]:
cols = ['PCI', 'ARFCN', 'BAND', 'RSSI', 'RSRP', 'RSRQ', 'SINR', 'CQI', 'TA']
for i in cols:
    jr[i] = pd.to_numeric(jr[i], errors='coerce')

In [10]:
jr_enb = jr.groupby(['xNBID', 'PCI']).agg({'RSSI':'mean', 'RSRP':'mean', 'RSRQ':'mean', 'SINR':'mean'}).sort_values(['xNBID', 'PCI'], ascending=True)

In [11]:
jr_enb

0                RSSI       RSRP       RSRQ      SINR
xNBID  PCI                                           
       313 -55.000000 -93.000000 -20.000000  1.400000
278920 365 -63.000000 -89.000000 -16.000000 -0.300000
279041 95  -63.000000 -85.187500 -14.375000 -0.300000
       123 -63.000000 -87.000000 -12.000000 -0.300000
279394 199 -55.000000 -87.000000 -13.000000  1.400000
       202 -55.000000 -85.285714 -12.571429  1.400000
279397 33  -61.581818 -81.972727 -11.545455  0.065455
       158 -58.304348 -90.956522 -12.086957  1.039130
279398 7   -55.000000 -80.571429 -13.000000  1.400000
       74  -55.000000 -93.294118 -17.882353  1.188235
279592 241 -55.000000 -85.900000 -13.200000  1.400000
       260 -56.882353 -85.882353 -10.235294  1.094118
279617 75  -63.000000 -89.444444 -13.833333 -0.300000
       332 -63.000000 -85.000000 -15.000000 -0.300000
279633 190 -55.000000 -85.800000 -13.400000  1.400000
279770 110 -63.000000 -88.000000 -10.000000  0.100000
280070 135 -55.000000 -89.631579 -10.894737  0.600000
280183 15  -68.600000 -94.800000 -13.500000  0.140000
       251 -69.363636 -97.045455  -9.045455  0.927273
280189 135 -55.000000 -75.424242  -4.666667  1.400000
       322 -55.000000 -81.400000  -8.900000  1.400000
280209 317 -55.000000 -77.588235 -11.882353  1.400000

In [12]:
jr_enb.style.highlight_null()

In [13]:
jr_enb.style.highlight_max(props='color:white;background-color:blue')

In [14]:
jr_enb.style.highlight_min(props='color:white;background-color:red')

In [15]:
jr_enb.style.highlight_between(left=-85, right=-70, props='color:white;background-color:lime')

In [16]:
jr_enb.style.bar()

In [17]:
jr_enb.style.background_gradient(cmap='RdYlGn', subset=['RSRP'])

In [18]:
def highlight_df(df, subset=None):
    df = df.style.highlight_max(props='color:blue;background-color:lightblue', subset=subset) \
           .highlight_min(props='color:red;background-color:lightpink', subset=subset) \
           .highlight_null(props='color:white;background-color:black', subset=subset)
    return df

In [19]:
highlight_df(jr_enb, subset=['RSRP'])